In [ ]:
!pip install datasets
!pip install peft
import pandas as pd
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset # Now this import should work
from peft import LoraConfig

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [ ]:
df1 = pd.read_csv("/content/dialogueText_file1.csv") # Read the CSV file into a DataFrame


In [ ]:
df = df1.sample(frac=0.4)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 97242 entries, 118747 to 55071
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   dialogueID  97242 non-null  object
 1   text        97242 non-null  object
dtypes: object(2)
memory usage: 2.2+ MB


In [ ]:
# Ensure the data has the necessary columns
assert 'dialogueID' in df.columns
assert 'text' in df.columns


In [ ]:
from sklearn.model_selection import train_test_split
df1 = train_test_split(df, test_size=0.2, random_state=42)

In [ ]:
 #Create a tokenizer
tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
!pip install datasets transformers
from datasets import Dataset
from transformers import AutoTokenizer



In [ ]:
# Prepare the dataset for fine-tuning
def prepare_dataset(batch):
    inputs = tokenizer(batch['text'],
                        truncation=True,
                        padding='max_length',
                        max_length=512,
                        return_attention_mask=True,
                        return_tensors='pt')
    labels = tokenizer(batch['text'],
                        truncation=True,
                        padding='max_length',
                        max_length=512,
                        return_attention_mask=True,
                        return_tensors='pt')
    # Do not flatten the tensors
    return {'input_ids': inputs['input_ids'],
            'attention_mask': inputs['attention_mask'],
            'labels': labels['input_ids']}

In [ ]:
!pip install datasets
from datasets import Dataset

# Convert your DataFrame to a Dataset object
dataset = Dataset.from_pandas(df)

# Now you can use the map function
dataset = dataset.map(prepare_dataset, batched=True)


Map:   0%|          | 0/97242 [00:00<?, ? examples/s]

In [ ]:
!pip install --upgrade transformers
from transformers import AutoModelForSeq2SeqLM # Import the correct class

# Load a model suitable for sequence-to-sequence tasks
model = AutoModelForSeq2SeqLM.from_pretrained('t5-small') # Use a model like T5

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 86.2 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.42.4
    Uninstalling transformers-4.42.4:
      Successfully uninstalled transformers-4.42.4


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
# Fine-tune the model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)

NameError: name 'torch' is not defined